In [1]:
import os
import numpy as np
import pandas as pd
from pyspark import SparkContext,SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [2]:
def CreateSparkContex():
    sparkconf=SparkConf().setAppName("shandongprocess").set("spark.ui.showConsoleProgress","false")
    sc=SparkContext(conf=sparkconf)
    print("master:"+sc.master)
    sc.setLogLevel("WARN")
    Setpath(sc)
    spark = SparkSession.builder.config(conf=sparkconf).getOrCreate()
    return sc,spark

In [3]:
def Setpath(sc):
    global Path
    if sc.master[:5]=="local":
        Path="file:/F:/software/work/Python/data"
    else:
        Path="hdfs://data"

In [4]:
sc,spark=CreateSparkContex()

master:local[*]


In [5]:
readdatapath=os.path.join(Path,'metadata_trans.csv')
print("Here we go!", readdatapath)
data=spark.read.csv(readdatapath,header=True)
# sc.stop()
# spark.stop()

Here we go! file:/F:/software/work/Python/data\metadata_trans.csv


In [6]:
data.createOrReplaceTempView("data")
data = spark.sql("SELECT * FROM data where subtime_trans < unsubtime_fill") # 数据清洗
data.createOrReplaceTempView("data_user")

In [7]:
data.take(1)

[Row(userid='18769452858', contractid='TV4KBY20Y', status='1', payway='903', price='20.0', subtime='20170402092917', unsubtime=None, endtime='20191205000000.0', subtime_trans='2017-04-02 09:29:17', unsubtime_trans=None, endtime_trans='2019-12-05 00:00:00', keeptime_days='976.6046643518519', unsubtime_fill='2019-12-05 00:00:00', effective_days='976.6046643518519', pro_class='电影', channel='电影', pro_name='魔百和电影连续包月', pack_type='连续包月', pack_type1='单产品包', pack_id='2', total_reve='660.0')]

# 产品 vs 购买时间

In [8]:
# 每年各产品的周订购情况
product_dict = {'产品类别': 'pro_class', '产品频道': 'channel', '产品包类型': 'pack_type'}
for key, product in product_dict.items():
    spark.sql("SELECT date_format(subtime_trans, 'yyyy') as subtime_year, %s, dayofweek(subtime_trans) as subtime_weekday, count(userid) FROM data_user group by subtime_year, %s, subtime_weekday order by subtime_year, %s, subtime_weekday asc" % (product, product, product)).toPandas().to_excel('./每年%s的周订购情况.xlsx' % (key), index=False)

In [9]:
# 每年各产品的月订购情况
product_dict = {'产品类别':'pro_class', '产品频道':'channel', '产品包类型':'pack_type'}
for key,product in product_dict.items():
    spark.sql("SELECT date_format(subtime_trans, 'yyyy') as subtime_year, %s, month(subtime_trans) as subtime_month, count(userid) FROM data_user group by subtime_year, %s, subtime_month order by subtime_year, %s, subtime_month asc" % (product, product, product)).toPandas().to_excel('./每年%s的月订购情况.xlsx' % (key),index=False)

In [10]:
# 每年各产品的日订购情况
product_dict = {'产品类别':'pro_class', '产品频道':'channel', '产品包类型':'pack_type'}
for key,product in product_dict.items():
    spark.sql("SELECT date_format(subtime_trans, 'yyyy') as subtime_year, %s, day(subtime_trans) as subtime_day, count(userid) FROM data_user group by subtime_year, %s, subtime_day order by subtime_year, %s, subtime_day asc" % (product, product, product)).toPandas().to_excel('./每年%s的日订购情况.xlsx' % (key),index=False)

# 产品 vs 退订时间

In [11]:
# 每年各产品的周订购情况
product_dict = {'产品类别':'pro_class', '产品频道':'channel', '产品包类型':'pack_type'}
for key,product in product_dict.items():
    spark.sql("SELECT date_format(unsubtime_trans, 'yyyy') as unsubtime_year, %s, dayofweek(unsubtime_trans) as unsubtime_weekday, count(userid) FROM data_user where unsubtime_trans is not NULL and unsubtime_trans <= endtime_trans group by unsubtime_year, %s, unsubtime_weekday order by unsubtime_year, %s, unsubtime_weekday asc" % (product, product, product)).toPandas().to_excel('./每年%s的周退订情况.xlsx' % (key),index=False)

In [12]:
# 每年各产品的月订购情况
product_dict = {'产品类别':'pro_class', '产品频道':'channel', '产品包类型':'pack_type'}
for key,product in product_dict.items():
    spark.sql("SELECT date_format(unsubtime_trans, 'yyyy') as unsubtime_year, %s, month(unsubtime_trans) as unsubtime_month, count(userid) FROM data_user where unsubtime_trans is not NULL and unsubtime_trans <= endtime_trans group by unsubtime_year, %s, unsubtime_month order by unsubtime_year, %s, unsubtime_month asc" % (product, product, product)).toPandas().to_excel('./每年%s的月退订情况.xlsx' % (key),index=False)

In [13]:
# 每年各产品的日订购情况
product_dict = {'产品类别':'pro_class', '产品频道':'channel', '产品包类型':'pack_type'}
for key,product in product_dict.items():
    spark.sql("SELECT date_format(unsubtime_trans, 'yyyy') as unsubtime_year, %s, day(unsubtime_trans) as unsubtime_day, count(userid) FROM data_user where unsubtime_trans is not NULL and unsubtime_trans <= endtime_trans group by unsubtime_year, %s, unsubtime_day order by unsubtime_year, %s, unsubtime_day asc" % (product, product, product)).toPandas().to_excel('./每年%s的日退订情况.xlsx' % (key),index=False)

# 退订意愿 vs 支付方式

In [16]:
# 不同支付方式与订购用户数的关系
spark.sql("SELECT date_format(unsubtime_trans, 'yyyy') as unsubtime_year, payway, count(DISTINCT userid) FROM data_user where unsubtime_trans is not NULL and unsubtime_trans <= endtime_trans group by unsubtime_year, payway").show(30)

+--------------+------+----------------------+
|unsubtime_year|payway|count(DISTINCT userid)|
+--------------+------+----------------------+
|          2018|   903|                529743|
|          2019|   903|                967517|
|          2017|   903|                 37099|
|          2016|   903|                   136|
|          2018|   701|                   540|
|          2018|    56|                  9196|
|          2019|   701|                  4514|
|          2019|    56|                 11170|
+--------------+------+----------------------+



In [15]:
# 不同支付方式与订购人次的关系
spark.sql("SELECT payway, count(userid) FROM data_user where unsubtime_trans is not NULL and unsubtime_trans <= endtime_trans group by payway").show(30)

+------+-------------+
|payway|count(userid)|
+------+-------------+
|   903|      2371974|
|   701|         6016|
|    56|        26767|
+------+-------------+



# Spark SQL测试

In [ ]:
# 每年不同status的订购情况
spark.sql("SELECT count(*) FROM data_user where unsubtime_trans is not NULL and unsubtime_trans <= endtime_trans").show(30)

In [ ]:
# contractid, status, payway, price, subtime_trans 
results = spark.sql("SELECT userid, contractid, payway, price/100 as yuan, keeptime_days, effective_days, cast(date_format(subtime_trans, 'yyyyMM') as int) as subtime_y, cast(date_format(subtime_trans, 'yyyyMMdd') as int) as subtime_date,  cast(date_format(unsubtime_trans, 'yyyyMMdd') as int) as unsubtime_date, cast(date_format(endtime_trans, 'yyyyMMdd') as int) as endtime_date FROM data_user")
results.createOrReplaceTempView("to_date_user")
results.show(10)

In [ ]:
for por_name in ['玩吧游戏','咪咕游戏','电竞风云','成长乐园','百灵K歌']:
        print(spark.sql("SELECT count(userid) FROM to_date_user where subtime_y = 201905 and contractid in (select pro_id from product where pack_type = '连续包月' and pro_name  like '%s') " % (por_name + '%')).toPandas().values[0][0])

In [ ]:
spark.sql("SELECT count(userid) FROM to_date_user where contractid like 'TVWBY%' and subtime_y = 201906 ").count()

In [ ]:
user_keeptime = [20190601, 20190701, 20190801, 20190901, 20191001, 20191101, 20191201] # 20190201, 20190301, 20190401, 20190501, 20190601, 20190701, 20190801, 20190901, 
for val in user_keeptime:
    print(val)
    for por_name in ['玩吧游戏','咪咕游戏','电竞风云','成长乐园','百灵K歌']:
        print(spark.sql("SELECT count(userid) FROM to_date_user where contractid in (select pro_id from product where pack_type = '连续包月' and pro_name  like '%s') and (subtime_y = 201905 and endtime_date > %d)" % (por_name + '%', val)).toPandas().values[0][0])

In [ ]:
user_keeptime_num = [0, 1, 2, 3, 4, 5, 6] # 生成各订购时间的日期条件
for i,val in enumerate(user_keeptime_num): # 枚举遍历，i为列表索引，val列表值
    print(val,'------------')
    for por_name in ['新东方TV学堂','魔百和国粹','玩吧游戏','NBA续包月','咪咕游戏','魔百和纪录片','海看鼎级剧场包月','魔百和动漫','魔百和电影','魔百和热剧','电竞风云','成长乐园','百灵K歌']:
        if i == len(user_keeptime_num)-1: # 处理边界值
            print(spark.sql("SELECT count(userid) FROM to_date_user where subtime_y ==2019 and keeptime_days > %s and  contractid in (select pro_id from product where pack_type = '连续包月' and pro_name  like '%s')" % (val*30, por_name + '%')).toPandas().values[0][0])
        else:
            print(spark.sql("SELECT count(userid) FROM to_date_user where subtime_y ==2019 and (keeptime_days > %s and keeptime_days <= %s) and  contractid in (select pro_id from product where pack_type = '连续包月' and pro_name  like '%s')" % (val*30, user_keeptime_num[i+1]*30, por_name + '%')).toPandas().values[0][0])

In [ ]:
user_order_date = [0, (1, 10), (11, 20), (21, 31)] # 生成各订购时间的日期条件
for val in user_order_date: # 枚举遍历，i为列表索引，val列表值
    print(val,'------------')
    for por_name in ['新东方TV学堂','魔百和国粹','玩吧游戏','NBA续包月','咪咕游戏','魔百和纪录片','海看鼎级剧场包月','魔百和动漫','魔百和电影','魔百和热剧','电竞风云','成长乐园','百灵K歌']:
        if type(val) is int: # 处理边界值
            print(spark.sql("SELECT count(userid) FROM to_date_user where subtime_y ==2019 and keeptime_days = %s and  contractid in (select pro_id from product where pack_type = '连续包月' and pro_name  like '%s')" % (val, por_name + '%')).toPandas().values[0][0])
        elif type(val) is tuple:
            print(spark.sql("SELECT count(userid) FROM to_date_user where subtime_y ==2019 and (keeptime_days >= %s and keeptime_days <= %s) and  contractid in (select pro_id from product where pack_type = '连续包月' and pro_name  like '%s')" % (val[0],val[1], por_name + '%')).toPandas().values[0][0])
        else:
            raise Exception('Value Error')

In [ ]:
user_order_times_num = [1, 2, 3, (4, 6), (7, 9), (10, float("inf"))] # 生成各个时间点的条件
for val in user_order_times_num:
    if type(val) is int: # 单条件判断
        user_id_data = order_times[order_times['订购次数'] == val]['userid'] # 根据订购次数，获取符合条件的用户ID
        column_name = '%s次' % val
    elif type(val) is tuple: 

In [ ]:
user_order_date = [0, 10, 20, 31] # 生成各订购时间的日期条件
for i,val in enumerate(user_order_date): # 枚举遍历，i为列表索引，val列表值
    print(val,'------------')
    for por_name in ['新东方TV学堂','魔百和国粹','玩吧游戏','NBA续包月','咪咕游戏','魔百和纪录片','海看鼎级剧场包月','魔百和动漫','魔百和电影','魔百和热剧','电竞风云','成长乐园','百灵K歌']:
#         print(por_name)
        if i == len(user_order_date)-1: # 处理边界值
            break
        else:
            print(spark.sql("SELECT count(userid) FROM to_date_user where subtime_y ==2019 and (unsubtime_date > %s and unsubtime_date <= %s) and  contractid in (select pro_id from product where pack_type = '连续包月' and pro_name  like '%s')" % (val,user_order_date[i+1], por_name + '%')).toPandas().values[0][0])

In [ ]:
user_order_date = [0, 10, 20, 31] # 生成各订购时间的日期条件
for i,val in enumerate(user_order_date): # 枚举遍历，i为列表索引，val列表值
    print(val,'------------')
    for por_name in ['新东方TV学堂','魔百和国粹','玩吧游戏','NBA续包月','咪咕游戏','魔百和纪录片','海看鼎级剧场包月','魔百和动漫','魔百和电影','魔百和热剧','电竞风云','成长乐园','百灵K歌']:
#         print(por_name)
        if i == len(user_order_date)-1: # 处理边界值
            break
        else:
            print(spark.sql("SELECT count(userid) FROM to_date_user where subtime_y ==2019 and (subtime_date > %s and subtime_date <= %s) and  contractid in (select pro_id from product where pack_type = '连续包月' and pro_name  like '%s')" % (val,user_order_date[i+1], por_name + '%')).toPandas().values[0][0])

In [ ]:
# contractid, status, payway, price, subtime_trans
results = spark.sql("SELECT userid, contractid, payway, price/100 as yuan, date_format(subtime_trans, 'yyyyMM') as subtime_date, date_format(endtime_trans, 'yyyyMM') as endtime_date FROM data_user")
results.createOrReplaceTempView("to_date_user")
results.show(5)

In [ ]:
a = spark.sql("SELECT * FROM to_date_user WHERE contractid NOT IN {0}".format(('TVDJFYLXBY29Y','TVMBHRJBY20Y','TV4KBY20Y','TV4KMBKDMBY10Y','TVMBHDMBY20Y',
                                      'TVMBHJLPBY15Y','TVMBHXQBBY10Y','TVXDFTVXTXBY20Y','TVMBHRJBY20Y','TVMBHDMBY20Y')))

In [ ]:
b = results.where(col("contractid").isin({'TVDJFYLXBY29Y','TVMBHRJBY20Y','TV4KBY20Y','TV4KMBKDMBY10Y','TVMBHDMBY20Y',
                                      'TVMBHJLPBY15Y','TVMBHXQBBY10Y','TVXDFTVXTXBY20Y','TVMBHRJBY20Y','TVMBHDMBY20Y'}))

In [ ]:
# results.groupBy("subtime_date").sum().sort("subtime_date").toPandas().to_excel('./收入.xlsx',index=False) # write.csv('./收入.csv')
b.show(10)

In [ ]:
from collections import defaultdict
user_num_dict = defaultdict(list) # 生成一个元素为列表的字典
product_a = ['TVMBHDYDBY25Y','TV4KBY20Y','TV4KBY20Y','TV4KBY20Y','TVMBHDY120LYB','TV4KBY20Y','TV4KBY20Y','TVMBHDYRJ199LYB']
product_b = ['TVMBHRJDBY25Y','TVMBHRJBY20Y','TVMBHRJBY20Y','TVMBHRJBY20Y','TVMBHRJ120LYB','TVMBHRJBY20Y','TVMBHRJBY20Y']
product_c = ['TVMBHDMDBY20Y','TV4KMBKDMBY10Y','TVMBHDMBY20Y','TV4KMBKDMBY10Y','TVMBHDMBY20Y','TV4KMBKDMBY10Y','TVMBHDMBY20Y',
             'TVMBHDM99LYB','TVMBHDM120LYB','TV4KMBKDMBY10Y','TVMBHDMBY20Y','TV4KMBKDMBY10Y','TVMBHDMBY20Y']
for year in ['2017', '2018', '2019']:
    print(year)
    for month in ['01','02','03','04','05','06','07','08','09','10','11','12']:
        datetime = year + '-' + month
        user_num_dict['月份'].append(datetime)
        # spark.sql("SELECT count(distinct('userid')) FROM to_date_user")
        # user_num_dict['有效去重用户数'].append(results.filter(results.subtime_date <= datetime).filter(results.endtime_date > datetime).distinct().count())
        total = 0
        
        revenue_a = a.filter(a.subtime_date == datetime).where(col("contractid").isin(product_c)).agg({"yuan": "sum"}).toPandas().values[0][0]
        revenue_b = b.filter(b.subtime_date <= datetime).filter(b.endtime_date >= datetime).where(col("contractid").isin(product_c)).agg(
            {"yuan": "sum"}).toPandas().values[0][0]
        if revenue_a is not None:
            total+=revenue_a
        if revenue_b is not None:
            total+=revenue_b
        print(total)
        user_num_dict['收入'].append(total)  

In [ ]:
from collections import defaultdict
user_num_dict = defaultdict(list) # 生成一个元素为列表的字典
for payway in ['903', '701', '56']:
    for year in ['2015', '2016', '2017', '2018', '2019']:
        user_num_dict['年份'].append(year)
        year_total = 0
        for month in ['01','02','03','04','05','06','07','08','09','10','11','12']:
            datetime = year + '-' + month
            month_total = 0
            revenue_a = a.filter(a.subtime_date == datetime).filter(a.payway == payway).agg({"yuan": "sum"}).toPandas().values[0][0]
            revenue_b = b.filter(b.subtime_date <= datetime).filter(b.endtime_date >= datetime).filter(a.payway == payway).agg({"yuan": "sum"}).toPandas().values[0][0]
            if revenue_a is not None:
                month_total += revenue_a
            if revenue_b is not None:
                month_total += revenue_b
            year_total += month_total
        print(year_total)
        user_num_dict[payway].append(year_total)

In [ ]:
from collections import defaultdict
user_num_dict = defaultdict(list) # 生成一个元素为列表的字典
for year in ['2015', '2016', '2017', '2018', '2019']:
    user_num_dict['年份'].append(year)
    year_total = 0
    for month in ['01','02','03','04','05','06','07','08','09','10','11','12']:
        datetime = year + '-' + month
        month_total = 0
        revenue_b = b.filter(b.subtime_date <= datetime).filter(b.endtime_date >= datetime).agg({"yuan": "sum"}).toPandas().values[0][0]
        if revenue_b is not None:
            month_total += revenue_b
        year_total += month_total
    print(year_total)
    user_num_dict[payway].append(year_total)

In [ ]:
print(user_num_dict)
# user_num_data=pd.DataFrame(user_num_dict) # 将字典转换为DataFrame，字典元素必须为列表类型
# user_num_data.to_excel('./各年不同订购方式的收入.xlsx',index=False)

In [ ]:
from collections import defaultdict
user_num_dict = defaultdict(list) # 生成一个元素为列表的字典
for year in ['2015', '2016', '2017', '2018', '2019']:
    print(year)
    for month in ['01','02','03','04','05','06','07','08','09','10','11','12']:
        datetime = year + '-' + month
        user_num_dict['月份'].append(datetime)
        # spark.sql("SELECT count(distinct('userid')) FROM to_date_user")
        # user_num_dict['有效去重用户数'].append(results.filter(results.subtime_date <= datetime).filter(results.endtime_date > datetime).distinct().count())
        total = 0
        revenue_a = a.filter(a.subtime_date == datetime).agg({"yuan": "sum"}).toPandas().values[0][0]
        revenue_b = b.filter(b.subtime_date <= datetime).filter(b.endtime_date >= datetime).agg({"yuan": "sum"}).toPandas().values[0][0]
        if revenue_a is not None:
            total+=revenue_a
        if revenue_b is not None:
            total+=revenue_b
        print(revenue_a,revenue_b,total)
        user_num_dict['收入'].append(total)  

In [ ]:
user_num_data=pd.DataFrame(user_num_dict) # 将字典转换为DataFrame，字典元素必须为列表类型
user_num_data.to_excel('./收入.xlsx',index=False)

In [ ]:
spark.udf.register("strLen", (str: String) => str.length())

In [ ]:
results.select("subtime").

In [ ]:
results.sort(results.userid.desc(), results.contractid.asc()).show(1000)